In [1]:
!pip install streamlit yfinance yahooquery numpy tensorflow scikit-learn joblib


Defaulting to user installation because normal site-packages is not writeable


In [3]:
import yfinance as yf
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import joblib

np.seterr(all="ignore")

def get_indian_stocks():
    return [
        "RELIANCE.NS", "TCS.NS", "HDFCBANK.NS", "INFY.NS", "ICICIBANK.NS",
        "HINDUNILVR.NS", "KOTAKBANK.NS", "SBIN.NS", "BAJFINANCE.NS", "ITC.NS",
        "TATASTEEL.NS", "AXISBANK.NS", "WIPRO.NS", "BHARTIARTL.NS", "LT.NS",
        "MARUTI.NS", "ASIANPAINT.NS", "NESTLEIND.NS", "TATAMOTORS.NS", "SUNPHARMA.NS",
        "M&M.NS", "ULTRACEMCO.NS", "INDUSINDBK.NS", "POWERGRID.NS", "HCLTECH.NS"
    ]

def get_stock_data(ticker):
    try:
        stock = yf.download(ticker, period="2y", interval="1d")
        if stock.empty:
            return None
        return stock["Close"].dropna().values.reshape(-1, 1)
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

tickers = get_indian_stocks()
all_data = []

for ticker in tickers:
    data = get_stock_data(ticker)
    if data is not None:
        all_data.append(data)

if len(all_data) > 0:
    data = np.concatenate(all_data, axis=0)
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)

    X_train, y_train = [], []
    for i in range(30, len(scaled_data)):
        X_train.append(scaled_data[i-30:i])
        y_train.append(scaled_data[i])

    X_train, y_train = np.array(X_train), np.array(y_train)

    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(30, 1)),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer="adam", loss="mean_squared_error")

    model.fit(X_train, y_train, epochs=15, batch_size=16, verbose=1)

    model.save("lstm_stock_model.keras")
    joblib.dump(scaler, "scaler.pkl")

    print("Model trained and saved successfully!")
else:
    print("No valid stock data available for training.")


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Epoch 1/15
772/772 ━━━━━━━━━━━━━━━━━━━━ 22s 24ms/step - loss: 0.0037
Epoch 2/15
772/772 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - loss: 6.4034e-04
Epoch 3/15
772/772 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 3.4112e-04
Epoch 4/15
772/772 ━━━━━━━━━━━━━━━━━━━━ 21s 27ms/step - loss: 1.6310e-04
Epoch 5/15
772/772 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 2.5425e-04
Epoch 6/15
772/772 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 2.4921e-04
Epoch 7/15
772/772 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - loss: 2.4196e-04
Epoch 8/15
772/772 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 1.2242e-04
Epoch 9/15
772/772 ━━━━━━━━━━━━━━━━━━━━ 19s 25ms/step - loss: 1.4669e-04
Epoch 10/15
772/772 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 2.7672e-04
Epoch 11/15
772/772 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - loss: 1.2541e-04
Epoch 12/15
772/772 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - loss: 1.5247e-04
Epoch 13/15
772/772 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - loss: 8.1297e-05
Epoch 14/15
772/772 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/s

In [6]:
import yfinance as yf
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import joblib
import datetime

model = load_model("lstm_stock_model.keras", compile=False)
model.compile(optimizer="adam", loss="mean_squared_error")
scaler = joblib.load("scaler.pkl")

def get_stock_data(ticker):
    stock = yf.download(ticker, period="31d", interval="1d")
    if stock.empty:
        print(f"Stock data for {ticker} not found!")
        return None
    return stock["Close"].dropna().values.reshape(-1, 1)

ticker = "TATASTEEL.NS" 
data = get_stock_data(ticker)

if data is not None and len(data) >= 30:
    scaled_data = scaler.transform(data)

    test_input = scaled_data[-30:].reshape(1, 30, 1)
    predicted_price = model.predict(test_input)

    predicted_price_float = float(predicted_price[0][0])
    predicted_price_actual = scaler.inverse_transform([[predicted_price_float]])[0][0]

    # Fetch today's actual price
    today_data = yf.download(ticker, period="1d", interval="1d")
    actual_price = today_data["Close"].values[0] if not today_data.empty else None

    today = datetime.date.today().strftime("%Y-%m-%d")

    print(f"Predicted Price for {today}: ₹{predicted_price_actual:.2f}")
    
    if actual_price:
        actual_price_float = float(actual_price)
        print(f"Actual Price for {today}: ₹{actual_price_float:.2f}")

        error = abs(actual_price_float - predicted_price_actual)
        print(f"Prediction Error: ₹{error:.2f}")
    else:
        print("Today's actual price is not available yet.")
else:
    print("Not enough data to make a prediction.")


[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 515ms/step


[*********************100%***********************]  1 of 1 completed

Predicted Price for 2025-02-21: ₹142.27
Actual Price for 2025-02-21: ₹140.76
Prediction Error: ₹1.51



C:\Users\Lenovo\AppData\Local\Temp\ipykernel_672\3636751894.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  actual_price_float = float(actual_price)


In [7]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Predict on training data
y_train_pred = model.predict(X_train)

# Convert predictions back to original scale
y_train_actual = scaler.inverse_transform(y_train.reshape(-1, 1))
y_train_pred_actual = scaler.inverse_transform(y_train_pred)

# Calculate performance metrics
mae = mean_absolute_error(y_train_actual, y_train_pred_actual)
mse = mean_squared_error(y_train_actual, y_train_pred_actual)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_train_actual - y_train_pred_actual) / y_train_actual)) * 100
r2 = r2_score(y_train_actual, y_train_pred_actual)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"R² Score: {r2:.4f}")


386/386 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Mean Absolute Error (MAE): 37.66
Mean Squared Error (MSE): 33838.28
Root Mean Squared Error (RMSE): 183.95
Mean Absolute Percentage Error (MAPE): 1.82%
R² Score: 0.9956
